In [ ]:
# Activar Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Calcula el pago de luz para tarifas residenciales.

import pandas as pd
import numpy as np

In [ ]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/"

# Escenarios.
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]
t = ["_tarifa_teorica", ""]

# Directorios.
path_cfe = path + "CFE/"
dirs = [RCP[1]]
csv = ".csv"
path_data = [path + "data/" + x + "/data." for x in dirs]
f_path = []
for p in path_data:
    f_path = ( f_path + [p + str(x) for x in fut]
        + [p + str(x) + "_tarifa_teorica" for x in fut]
        + [p + str(x) + "_pob_pib" for x in fut] )
f_path = [ path + "data/Actual/data" + x for x in t ] + f_path

f_tarifa = "Tarifa_1.csv"
f_dac = "DAC.csv"
f_regiones = "Regiones.csv"

tarifa = pd.read_csv(path_cfe + f_tarifa)
dac = pd.read_csv(path_cfe + f_dac)
regiones = pd.read_csv(path_cfe + f_regiones)

# Tipos de escalones para las tarifas.
escalon_0 = tarifa["Escalon"].unique()

# Se establece la tarifa DAC para cada estado.
regiones[ ["DAC Propuesto", "Equilibrio"] ] = np.nan
for i in regiones.itertuples():
    regiones.loc[ i[0], ["DAC 04-2013", "Equilibrio"] ] = ( 
        dac.drop(8).loc[ dac["Region"] == i.Region,
        ["DAC 04-2013", "Equilibrio"] ].values[0] )

# Impuestos y cargos fijos.    
IVA = 1.16
#FIJO_DAC = 121.75
FIJO_DAC = 78.85
MULT = 0.86

In [ ]:
# Pago actual

cons = ["Verano_Consumo_1*", "Verano_Consumo_DAC",
    "Invierno_Consumo_1*", "Invierno_Consumo_DAC"]

# Calculamos el pago para la tarifa
# teórica y para la tarifa real.
for k in [0, 1]:
    # Información mensual.
    df_0 = pd.read_csv(f_path[k] + csv)
    # Información anual.
    df_y = pd.read_csv(f_path[k] + "_yearly" + csv)

    # Promediamos el consumo y usuarios anuales.
    df = df_y.groupby("CVE_INEGI").mean()
    df["Tarifa"] = ""

    # Recuperamos la información de las tarifas.
    df[ [ "Tarifa", "NOM_ENT", "NOM_MUN" ] ] = (
        df_y[df_y["Año"] == 2010]
        .set_index("CVE_INEGI")[ 
        [ "Tarifa", "NOM_ENT", "NOM_MUN" ] ] )
    col = [x for x in df_y.columns]
    col.remove("CVE_INEGI")
    df = df[ col ]
    # Permite una tarifa diferente para cada año.
    #for i in df.itertuples():
    #    df.loc[i[0], "Tarifa"] = df_y.query(
    #        "CVE_INEGI==" + str(i[0]) +
    #        " & Año==2010" )["Tarifa"].values[0]

    # Redondeamos la cantidad de usuarios.
    df[["Usuarios_1*", "Usuarios_DAC"]] = df[
        ["Usuarios_1*", "Usuarios_DAC"]].round()

    # Retiramos la columna de año, que ya no tiene sentido.
    df.drop(columns = "Año", inplace = True)

    # Sumamos el consumo correspondiente al verano.
    df_2 = df_0.where(df_0["Verano"] == True)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum() / 7
    df[cons[0:2]] = df_2[["Consumo_1*", "Consumo_DAC"]]

    # Sumamos el consumo correspondiente al invierno.
    df_2 = df_0.where(df_0["Verano"] == False)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum() / 7
    df[cons[2:4]] = df_2[["Consumo_1*", "Consumo_DAC"]]

    # Retiramos las columnas de consumo total anual.
    df.drop(columns = [
    #"Consumo_1*",
    "Consumo_DAC"
    ], inplace = True)

    # Unimos las tarifas 1 y DAC.
    df["Verano_Consumo_1*"] += df["Verano_Consumo_DAC"]
    df.drop("Verano_Consumo_DAC", axis = 1, inplace = True)
    df["Invierno_Consumo_1*"] += df["Invierno_Consumo_DAC"]
    df.drop("Invierno_Consumo_DAC", axis = 1, inplace = True)
    df["Usuarios_1*"] += df["Usuarios_DAC"]
    df.drop("Usuarios_DAC", axis = 1, inplace = True)

    df.reset_index(inplace = True)

    # Se crean columnas para los
    # escalones de verano e invierno.
    verano_escalon = []
    invierno_escalon = []
    for e in escalon_0:
        verano_escalon.append("Verano_" + e)
        invierno_escalon.append("Invierno_" + e)
    escalon = [verano_escalon,
        invierno_escalon]
    df[verano_escalon] = np.nan
    df[invierno_escalon] = np.nan

    # Se establece el valor de consumo límite para cada escalon.
    for i in df.itertuples():
        df.loc[i[0], invierno_escalon + verano_escalon] = tarifa.loc[
            tarifa["Tarifa"] == i.Tarifa, "Límite_p_2"].array * 6

    # Columnas para consumo en verano e invierno.
    estacion = ["Verano_", "Invierno_"]
    verano_escalon_cons = [ "Cons_" +
        x for x in verano_escalon ]
    invierno_escalon_cons = [ "Cons_" +
        x for x in invierno_escalon ]
    escalon_cons = [ verano_escalon_cons,
        invierno_escalon_cons ]

    # Columnas para verano.
    df["Verano_Consumo_Usuario"] = ( df["Verano_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[0]] = df[escalon[0]].copy()
    df[estacion[0] + "Pago"] = 0

    # Columnas para invierno.
    df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[1]] = df[escalon[1]].copy()
    df[estacion[1] + "Pago"] = 0

    df["DAC_Pago"] = 0

    # Iteramos para verano e invierno.
    for e in [0, 1]:

        # Offset de columnas de verano e invierno.
        if e == 0: s = 0
        else: s = 6

        col = np.concatenate( [
            [estacion[e] + "Consumo_Usuario"],
            escalon_cons[e]
            ] )

        # Calculamos los escalones de consumo
        # de acuerdo con cada límite.
        for i in range(len(col[:-1])):
            df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

        # Eliminamos valores negativos.
        df[col[1:]] = df[col[1:]].clip( lower = 0 )

        col = np.concatenate( [ escalon_cons[e] ] )
        col = np.flip(col)

        # Recorremos los escalones a su posición adecuada.
        for i in range(len(col[:-1])):
            df[col[i]] = df[col[i + 1]]

        for i in df.itertuples():
            
            # No se está en tarifa DAC.
            if ( ( i.Verano_Consumo_Usuario +
                i.Invierno_Consumo_Usuario )
                <= ( i.Verano_Excedente * 2 ) ):
                # Si el consumo es menor al límite básico, el
                # escalon básico es igual al consumo total.
                if i[40 + s] == 0:
                    df.loc[i[0], escalon_cons[e][0]] = i[39 + s]
                # Si el consumo excede un escalón, 
                # el consumo de un escalón es su límite.
                for j in range(3):
                    if i[40 + s + j + 1] > 0:
                        df.loc[i[0], escalon_cons[e][j]] = i[
                            31 + s * 4 // 6 + j]
            
                df.loc[i[0], escalon_cons[e]] *= tarifa.query(
                  "Tarifa=='" + i.Tarifa + "' & Temporada=='"
                  + estacion[e][:-1] + "'")["Precio Propuesto_2"].array
        
            # Sí está en tarifa DAC.
            else:
                # El resto de pagos es igual a cero.
                df.loc[i[0], col] = 0

                # Se calcula el pago con todo el consumo general.
                df.loc[i[0], "DAC_Pago"] = ( (
                    i.Verano_Consumo_Usuario +
                    i.Invierno_Consumo_Usuario )
                    * regiones.loc[
                    regiones["CVE_ENT"] == i.CVE_ENT,
                    "DAC 04-2013"].values[0] + FIJO_DAC )
            
                #df.loc[i[0], "Tarifa"] = "DAC"

        # Se calcula el pago por temporada.
        df[estacion[e] + "Pago"] = ( df[
            np.concatenate( [ escalon_cons[e]
            ] ) ].sum(axis = 1) * IVA )

    # Se eliminan las columnas de apoyo.
    df.drop( columns = np.concatenate( [
        escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
        ["Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
        ] ), inplace = True )

    # Se calcula el pago total.
    df["Pago_real"] = ( df[[estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago"]].sum(axis = 1) )

    # Se eliminan los pagos parciales.
    df.drop( columns = [ estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago" ], inplace = True )

    # Unimos el consumo de verano e invierno.
    df["Consumo_1*"] = ( df["Verano_Consumo_1*"]
        + df["Invierno_Consumo_1*"] )
    df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
    df["Pago_sin_subsidio"] = np.nan

    # Calculamos el precio sin subsidio, es decir,
    # con un precio plano igual al precio marginal.
    for i in df.itertuples():
        df.loc[i[0], "Pago_sin_subsidio"] = ( 
            i.Consumo_Usuario * regiones.loc[
            regiones["CVE_ENT"] == i.CVE_ENT,
            "Equilibrio" ].values[0] * IVA * MULT )

    # Se calcula el subsidio.
    df["Subsidio_Usuario"] = ( df["Pago_sin_subsidio"] -
        df["Pago_real"] )
    df["Subsidio"] = df["Subsidio_Usuario"] * df["Usuarios_1*"]

    df.to_csv(f_path[k] + "_Pago_Propuesto_2" + csv, index = False)

    print(f_path[k] + "_Pago_Propuesto_2" + csv)
    print(f"Pago    : {df['Pago_real'].sum():e}")
    print(f"Subsidio: {df['Subsidio'].sum():e}")
    print(f"Usuarios: {df['Usuarios_1*'].sum():e}")
    print(f"Consumo : {df['Consumo_1*'].sum():e}\n")  

df

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/Actual/data_tarifa_teorica_Pago_Propuesto_2.csv
Pago    : 4.721379e+06
Subsidio: 2.449874e+10
Usuarios: 3.322479e+07
Consumo : 5.323831e+10

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/Actual/data_Pago_Propuesto_2.csv
Pago    : 4.647085e+06
Subsidio: 2.691754e+10
Usuarios: 3.322479e+07
Consumo : 5.323831e+10



,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Tarifa,lon,lat,...,$luz,$GLP,Area,Verano_Consumo_1*,Invierno_Consumo_1*,Pago_real,Consumo_Usuario,Pago_sin_subsidio,Subsidio_Usuario,Subsidio
0,1001,1.0,1.0,Aguascalientes,Aguascalientes,3.094919e+08,254974.0,1,-102.295872,21.811400,...,NaN,11.211926,116635.924759,1.605032e+08,1.489886e+08,2539.979048,1213.817321,3215.622943,675.643894,1.722716e+08
1,1002,1.0,2.0,Aguascalientes,Asientos,1.243081e+07,11557.0,1,-102.045590,22.126479,...,NaN,11.211926,54337.426603,6.447211e+06,5.983595e+06,2057.472909,1075.608401,2849.482364,792.009454,9.153253e+06
2,1003,1.0,3.0,Aguascalientes,Calvillo,1.809181e+07,16736.0,1,-102.704911,21.900640,...,NaN,11.211926,92335.783229,9.382205e+06,8.709606e+06,2076.336512,1081.011686,2863.796649,787.460137,1.317893e+07
3,1004,1.0,4.0,Aguascalientes,CosÃ­o,4.038387e+06,3788.0,1,-102.297038,22.360619,...,NaN,11.211926,12838.354454,2.094395e+06,1.943991e+06,2024.277579,1066.099940,2824.292721,800.015142,3.030457e+06
4,1005,1.0,5.0,Aguascalientes,JesÃºs MarÃ­a,3.476588e+07,30597.0,1,-102.445700,21.932112,...,NaN,11.211926,49989.509158,1.802838e+07,1.673750e+07,2269.184880,1136.251117,3010.135954,740.951074,2.267088e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32.0,54.0,Zacatecas,Villa Hidalgo,3.691307e+06,3923.0,1,-101.714324,22.407345,...,NaN,11.268627,37142.598662,2.025577e+06,1.665730e+06,1587.326529,940.939806,2451.615816,864.289287,3.390607e+06
2465,32055,32.0,55.0,Zacatecas,Villanueva,8.254025e+06,8847.0,1,-102.851571,22.298130,...,NaN,11.268627,215650.560221,4.530459e+06,3.723565e+06,1559.518349,932.974438,2430.862076,871.343727,7.708778e+06
2466,32056,32.0,56.0,Zacatecas,Zacatecas,4.321439e+07,37702.0,1,-102.678055,22.735730,...,NaN,11.268627,43682.101050,2.368850e+07,1.952589e+07,2303.950936,1146.209496,2986.445374,682.494438,2.573141e+07
2467,32057,32.0,57.0,Zacatecas,Trancoso,2.057456e+07,15674.0,1,-102.312009,22.755625,...,NaN,11.268627,21844.700672,1.134779e+07,9.226769e+06,2885.035835,1312.655289,3420.119383,535.083548,8.386900e+06


In [ ]:
# Pago futuro

names = ["", "_tarifa_teorica", "pob_pib"]

#for k in range(1, len(f_path)):
for k in range(2, len(f_path)):

    cons = ["Verano_Consumo_1*", "Verano_Consumo_DAC",
        "Invierno_Consumo_1*", "Invierno_Consumo_DAC"]

    # Información mensual.
    df_0 = pd.read_csv(f_path[k] + csv)
    # Información anual.
    df = pd.read_csv(f_path[k] + "_yearly" + csv)
    df.set_index("CVE_INEGI", inplace = True)
    df["Usuarios_1*"] = df_0.groupby(
        "CVE_INEGI").mean()["Usuarios_1*"]
    #df.drop(columns = ["NOM_ENT", "NOM_MUN"], inplace = True)

    # Redondeamos la cantidad de usuarios.
    #df[["Usuarios_1*", "Usuarios_DAC"]] = df[
    #    ["Usuarios_1*", "Usuarios_DAC"]].round()
    df["Usuarios_1*"] = df["Usuarios_1*"].round()

    # Retiramos la columna de año, que ya no tiene sentido.
    df.drop(columns = "Año", inplace = True)

    # Sumamos el consumo correspondiente al verano.
    df_2 = df_0.where(df_0["Verano"] == True)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum()
    #df[cons[0:2]] = df_2[["Consumo_1*", "Consumo_DAC"]]
    df[cons[0]] = df_2["Consumo_1*"]

    # Sumamos el consumo correspondiente al invierno.
    df_2 = df_0.where(df_0["Verano"] == False)
    df_2 = df_2.groupby(["CVE_INEGI"]).sum()
    #df[cons[2:4]] = df_2[["Consumo_1*", "Consumo_DAC"]]
    df[cons[2]] = df_2["Consumo_1*"]

    # Retiramos las columnas de consumo total anual.
    df.drop(columns = [
    #    "Consumo_1*",
        "Consumo_DAC"
        ], inplace = True)

    # Unimos las tarifas 1 y DAC.
    #df["Verano_Consumo_1*"] += df["Verano_Consumo_DAC"]
    #df.drop("Verano_Consumo_DAC", axis = 1, inplace = True)
    #df["Invierno_Consumo_1*"] += df["Invierno_Consumo_DAC"]
    #df.drop("Invierno_Consumo_DAC", axis = 1, inplace = True)
    #df["Usuarios_1*"] += df["Usuarios_DAC"]
    df.drop("Usuarios_DAC", axis = 1, inplace = True)

    df.reset_index(inplace = True)
        
    # Se crean columnas para los
    # escalones de verano e invierno.
    verano_escalon = []
    invierno_escalon = []
    for e in escalon_0:
        verano_escalon.append("Verano_" + e)
        invierno_escalon.append("Invierno_" + e)
    escalon = [verano_escalon,
        invierno_escalon]
    df[verano_escalon] = np.nan
    df[invierno_escalon] = np.nan
        
    # Se establece el valor de consumo límite para cada escalon.
    for i in df.itertuples():
        df.loc[i[0], invierno_escalon + verano_escalon] = tarifa.loc[
            tarifa["Tarifa"] == i.Tarifa, "Límite_p_2"].array * 6

    # Columnas para consumo en verano e invierno.
    estacion = ["Verano_", "Invierno_"]
    verano_escalon_cons = [ "Cons_" +
        x for x in verano_escalon ]
    invierno_escalon_cons = [ "Cons_" +
        x for x in invierno_escalon ]
    escalon_cons = [ verano_escalon_cons,
        invierno_escalon_cons ]

    # Columnas para verano.
    df["Verano_Consumo_Usuario"] = ( df["Verano_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[0]] = df[escalon[0]].copy()
    df[estacion[0] + "Pago"] = 0

    # Columnas para invierno.
    df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
        / df["Usuarios_1*"] )
    df[escalon_cons[1]] = df[escalon[1]].copy()
    df[estacion[1] + "Pago"] = 0

    df["DAC_Pago"] = 0

    # Iteramos para verano e invierno.
    for e in [0, 1]:

        # Offset de columnas de verano e invierno.
        if e == 0: s = 0
        else: s = 6

        col = np.concatenate( [
            [estacion[e] + "Consumo_Usuario"],
            escalon_cons[e]
            ] )

        # Calculamos los escalones de consumo
        # de acuerdo con cada límite.
        for i in range(len(col[:-1])):
            df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

        # Eliminamos valores negativos.
        df[col[1:]] = df[col[1:]].clip( lower = 0 )

        col = np.concatenate( [ escalon_cons[e] ] )
        col = np.flip(col)

        # Recorremos los escalones a su posición adecuada.
        for i in range(len(col[:-1])):
            df[col[i]] = df[col[i + 1]]

        for i in df.itertuples():
        
            # No se está en tarifa DAC.
            if ( ( i.Verano_Consumo_Usuario +
                i.Invierno_Consumo_Usuario )
                <= ( i.Verano_Excedente * 2 ) ):
                # Si el consumo es menor al límite básico, el
                # escalon básico es igual al consumo total.
                if i[40 + s] == 0:
                    df.loc[i[0], escalon_cons[e][0]] = i[39 + s]
                # Si el consumo excede un escalón, 
                # el consumo de un escalón es su límite.
                for j in range(3):
                    if i[40 + s + j + 1] > 0:
                        df.loc[i[0], escalon_cons[e][j]] = i[
                            31 + s * 4 // 6 + j]
                    
                df.loc[i[0], escalon_cons[e]] *= tarifa.query(
                    "Tarifa=='" + i.Tarifa + "' & Temporada=='"
                    + estacion[e][:-1] + "'")["Precio Propuesto_2"].array
    
            # Sí está en tarifa DAC.
            else:
                # El resto de pagos es igual a cero.
                df.loc[i[0], col] = 0

                # Se calcula el pago con todo el consumo general.
                df.loc[i[0], "DAC_Pago"] = ( (
                    i.Verano_Consumo_Usuario +
                    i.Invierno_Consumo_Usuario )
                    * regiones.loc[
                    regiones["CVE_ENT"] == i.CVE_ENT,
                        "DAC 04-2013"].values[0] + FIJO_DAC )
                    
                #df.loc[i[0], "Tarifa"] = "DAC"

        # Se calcula el pago por temporada.
        df[estacion[e] + "Pago"] = ( df[
            np.concatenate( [ escalon_cons[e]
            ] ) ].sum(axis = 1) * IVA )

    # Se eliminan las columnas de apoyo.
    df.drop( columns = np.concatenate( [
        escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
        ["Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
        ] ), inplace = True )

    # Se calcula el pago total.
    df["Pago_real"] = ( df[[estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago"]].sum(axis = 1) )

    # Se eliminan los pagos parciales.
    df.drop( columns = [ estacion[0] + "Pago",
        estacion[1] + "Pago", "DAC_Pago" ], inplace = True )

    # Unimos el consumo de verano e invierno.
    df["Consumo_1*"] = ( df["Verano_Consumo_1*"]
        + df["Invierno_Consumo_1*"] )
    df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
    df["Pago_sin_subsidio"] = np.nan

    # Calculamos el precio sin subsidio, es decir,
    # con un precio plano igual al precio marginal.
    for i in df.itertuples():
        df.loc[i[0], "Pago_sin_subsidio"] = ( 
            i.Consumo_Usuario * regiones.loc[
            regiones["CVE_ENT"] == i.CVE_ENT,
            "Equilibrio" ].values[0] * IVA * MULT )

    # Se calcula el subsidio.
    df["Subsidio_Usuario"] = ( df["Pago_sin_subsidio"]
        - df["Pago_real"] )
    df["Subsidio"] = df["Subsidio_Usuario"] * df["Usuarios_1*"]

    df.to_csv( f_path[k] + "_Pago_Propuesto_2" 
        + csv, index = False )

    print(f_path[k] + "_Pago_Propuesto_2" + csv)     
    print(f"Subsidio: {df.Subsidio.sum():e}")
    print(f"Usuarios: {df['Usuarios_1*'].sum():e}")
    print(f"Consumo : {df['Consumo_1*'].sum():e}\n")        

df

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2030_Pago_Propuesto_2.csv
Subsidio: 3.846428e+10
Usuarios: 4.674058e+07
Consumo : 7.759272e+10

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2050_Pago_Propuesto_2.csv
Subsidio: 4.130272e+10
Usuarios: 5.871302e+07
Consumo : 1.124139e+11

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2070_Pago_Propuesto_2.csv
Subsidio: 3.677225e+10
Usuarios: 6.239606e+07
Consumo : 1.417807e+11

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2030_tarifa_teorica_Pago_Propuesto_2.csv
Subsidio: 3.627508e+10
Usuarios: 4.674058e+07
Consumo : 7.759272e+10

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2050_tarifa_teorica_Pago_Propuesto_2.csv
Subsidio: 4.002322e+10
Usuarios: 5.871302e+07
Consumo : 1.124139e+11

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data.2070_tarifa_teorica_Pago_Propuesto_2.csv
Subsidio: 3.629430e+10
Usuarios: 6.239606

,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Tarifa,lon,lat,...,$luz,$GLP,Area,Verano_Consumo_1*,Invierno_Consumo_1*,Pago_real,Consumo_Usuario,Pago_sin_subsidio,Subsidio_Usuario,Subsidio
0,1001,1,1,Aguascalientes,Aguascalientes,608878096.0,280328.0,1A,-102.295872,21.811400,...,NaN,10.151,116635.924759,3.154210e+08,2.934571e+08,5602.258779,2172.020262,5754.076880,151.818101,4.255886e+07
1,1002,1,2,Aguascalientes,Asientos,58753767.0,30330.0,1A,-102.045590,22.126479,...,NaN,10.151,54337.426603,3.063941e+07,2.811436e+07,4782.295616,1937.150247,5131.863476,349.567860,1.060239e+07
2,1003,1,3,Aguascalientes,Calvillo,79059454.5,41698.0,1A,-102.704911,21.900640,...,NaN,10.151,92335.783229,4.133197e+07,3.772748e+07,4638.638399,1896.001115,5022.851938,384.213539,1.602094e+07
3,1004,1,4,Aguascalientes,CosÃ­o,38732507.8,22120.0,1A,-102.297038,22.360619,...,NaN,10.151,12838.354454,2.020633e+07,1.852617e+07,4132.480991,1751.017532,4638.764045,506.283053,1.119898e+07
4,1005,1,5,Aguascalientes,JesÃºs MarÃ­a,128153462.0,61151.0,1A,-102.445700,21.932112,...,NaN,10.151,49989.509158,6.701152e+07,6.114194e+07,5335.775047,2095.688738,5551.860784,216.085737,1.321386e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32,54,Zacatecas,Villa Hidalgo,24598189.7,16791.0,1A,-101.714324,22.407345,...,NaN,10.129,37142.598662,1.302888e+07,1.156931e+07,3133.824879,1464.962760,3816.956037,683.131158,1.147046e+07
2465,32055,32,55,Zacatecas,Villanueva,21327846.6,13555.0,1A,-102.851571,22.298130,...,NaN,10.129,215650.560221,1.139100e+07,9.936845e+06,3512.499526,1573.430218,4099.567670,587.068144,7.957709e+06
2466,32056,32,56,Zacatecas,Zacatecas,159720598.0,80585.0,1A,-102.678055,22.735730,...,NaN,10.129,43682.101050,8.236036e+07,7.736024e+07,4938.921070,1982.013998,5164.131472,225.210403,1.814858e+07
2467,32057,32,57,Zacatecas,Trancoso,22201033.7,12315.0,1A,-102.312009,22.755625,...,NaN,10.129,21844.700672,1.174570e+07,1.045533e+07,4313.133544,1802.763597,4697.095097,383.961553,4.728487e+06
